### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [23]:
import sys
sys.path.append('../src')

import requests
import re

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [ ]:
# Series stored as a dictionary
series = {'CUUR0000SA0': 'All items',
          'CUUR0000SAM': 'Medical care',
          'CUUR0000SAH': 'Housing',
          'CUUR0000SEFV': 'Food away from home',
          'CUUR0000SAF11': 'Food at home',
          'CUUR0000SAR': 'Recreation',
          'CUUR0000SAE1': 'Education',
          'CUUR0000SAT': 'Transportation',
          'CUUR0000SAA': 'Apparel',
          'CUUR0000SA0E': 'Energy',
          'CUUR0000SAE2': 'Communication'}

In [60]:
url = 'https://www.bls.gov/cpi/tables/relative-importance/2018.txt'
rel_imp = pd.read_csv(url, header=None, delim_whitespace=True)
r = requests.get(url)
data = r.text

wgts = {value: re.findall(value+r'\..*\s+(\d+\.\d+)\s+\d+\.\d+\s+\r', data)[0] 
        for key, value in series.items()}

In [ ]:
# Start year and end year
years = (2017, 2019)
df = bls_api(series, years, bls_key)

In [117]:
wgt_date = pd.to_datetime('2018-12-01')
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

In [118]:
d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}

In [120]:
results = pd.DataFrame(d).drop('All items').round(2).sort_values('Latest', ascending=False)
results.to_csv(data_dir / 'cpi_comp.csv', index_label='name')

mo1  = df.index[-1].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo1.txt', mo1)

mo2  = df.index[-13].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo2.txt', mo2)